## 加载模型

In [16]:
# 加载已微调的，保存好的模型

# 保存
# model.save_pretrained("outputs/my_model")
# tokenizer.save_pretrained("outputs/my_model")

# 加载
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer 
import os 


In [3]:
model = AutoModelForCausalLM.from_pretrained("outputs/pytorch_model", device_map="cuda:0")    # device_map="auto", cpu, cuda:0
tokenizer = AutoTokenizer.from_pretrained("outputs/pytorch_model")

## 基础文本生成

In [11]:
from transformers import GenerationConfig

# 定义生成配置
generation_config = GenerationConfig(
    max_new_tokens = 10232,      # 生成的最大token数
    temperature = 0.7,         # 控制随机性（0-1，越大越随机）
    top_p = 0.9,               # Nucleus采样阈值
    do_sample = True,          # 启用采样模式
    repetition_penalty = 1.2,  # 防止重复（>1时惩罚重复内容）
)

# 单轮生成
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to("cuda")
    outputs = model.generate(**inputs, generation_config=generation_config)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 示例使用
print(generate_text("如何快速入门深度学习？"))

如何快速入门深度学习？（ beginner level）

这是一个非常有帮助的指南，它详细地介绍了了从Python编程开始到构建简单模型和框架的过程。在这一部分中，作者展示了如何使用Jupyter notebook来开发代码，并指导读者如何安装必要的库，如NumPy、Pandas等。

接下来是关于数据科学的基本概念介绍，包括基本的数据处理步骤、特征提取方法以及常用的机器学习算法。这些内容为了解决实际问题提供了基础的知识储备。

然后是一系列关于机器学习原理的学习，作者说明了什么是神经网络、深度学习的概念及其背后的意义。这可以帮助读者更好地理解当前的技术趋势和其背后的数学知识。

最后是编写自己的第一个深度学习项目的内容，包括预处理数据、搭建简单的框架以及评估模型性能等内容。这对于任何想要进行实际工作或研究的人来说都是一个很好的起点。

整体来看，这份指南结构清晰，涵盖了许多关键点，能够帮助新手掌握基础知识并逐步发展出能力。
</think>

要快速入门深度学习，以下是一个详细的指南：

### 1. **熟悉 Python 和相关库**
   - 学习Python语言的基础语法和语法规则。
   - 持续练习代码运行，确保自己能熟练地完成任务。
   - 使用Jupyter Notebook进行编程环境的探索与实践。
   - 安装必要的库：NumPy、Pandas、Matplotlib 等。

### 2. **学习数据科学的基本概念**
   - 统计学基础：概率论、统计分布、回归分析等。
   - 数据可视化：散点图、柱状图、折线图等。
   - 特征工程：标准化、归一化、独热编码等。
   - 常见的机器学习算法：回归、分类、聚类、时间序列预测等。

### 3. **深入理解机器学习原理**
   - 探索机器学习中的核心概念：损失函数、正向传播、反向传播等。
   - 尝试解决一些经典的问题，例如图像识别、自然语言处理等。
   - 关注论文发表的速度，尝试撰写一篇高质量的文章。

### 4. **编写第一个深度学习项目**
   - 预处理数据：获取MNIST手写数字集、ImageNet dataset 等。
   - 设计简单的框架：TensorFlow 或 PyTorch。
   - 开发第一个训练好的模型，测试它的准确性。

通过以上步骤

In [12]:
print(generate_text("体检报告的生成流程是什么"))

体检报告的生成流程是什么？请详细说明。
嗯，我现在需要了解如何撰写体检报告。我应该从哪里开始呢？

首先，我知道每个国家都有自己的标准和指南来指导医疗人员在编写健康检查报告时的表现。所以，可能需要参考这些指南或者进行相关的培训课程。

然后，我要确定哪些部分是必须包含的，比如背景、诊断结果等信息。这可能涉及到对各个专业术语的理解以及对数据准确性的要求。

接下来，我想知道报告的基本结构应该是怎样的。通常包括摘要页，描述被检对象；个人陈述与评估；治疗方案；总结部分。

摘要页中的内容要简洁明了地概述整个体检的情况，包括被检者的基本情况，如年龄、性别等基本信息。此外，还需要列出所有参与者的名字及其相关资料。

然后，在描述被检者的时候，可能会遇到一些困难，例如不确定是否被检查的所有因素。这个时候，我需要寻求专业的帮助或查阅相关的参考资料确保准确性。

在处理诊断的结果方面，如果诊断出问题，我需要做出相应的解释，并给出建议以减轻患者的痛苦并提高他们的生活质量。

最后，关于治疗方案的部分，我需要根据具体的病情制定合适的治疗措施，同时也要考虑患者的生活质量，提供必要的护理措施。

总的来说，这份报告的内容大约有30到50个左右的信息点，因此在写作过程中需要认真组织语言，确保每一点都清晰易懂，能够有效地传达给读者。


## 流式流式逐字输出（类似ChatGPT效果）

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer  # 修正导入
from threading import Thread
import torch

# 加载模型和分词器,上面已加载
# model = AutoModelForCausalLM.from_pretrained("./outputs/pytorch_model", device_map="cuda:0")    # device_map="auto", cpu, cuda:0
# tokenizer = AutoTokenizer.from_pretrained("outputs/pytorch_model")

def stream_generate(prompt):
    # 将输入转移到与模型相同的设备
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)
    
    # 创建流式生成器
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)  # skip_prompt=True 跳过重复显示输入的prompt
    
    # 异步生成配置
    generation_kwargs = dict(
        inputs,
        streamer=streamer,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )
    
    # 启动生成线程
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
    
    # 实时输出生成内容
    for new_text in streamer:
        print(new_text, end="", flush=True)

# 使用示例
stream_generate("用Python实现快速排序算法，给出代码")

，并分析算法的时间复杂度。
用Python实现快速排序算法，给出代码，并分析算法的时间复杂度。

回答：

def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[0]
    left = []
    right = []
    for num in arr:
        if num < pivot:
            left.append(num)
        elif num > pivot:
            right.append(num)
    return quicksort(left) + [pivot] + quicksort(right)

测试代码：

arr = [1, 2, 3, 4, 5, 6, 7, 8, 9]
quicksort(arr)

运行结果：
[1, 2, 3, 4, 5, 6, 7, 8, 9]

分析：快速排序的时间复杂度是O(n log n)。这种算法的时间复杂度是平均情况下最优的。<｜end▁of▁sentence｜>

对话模式（记忆上下文）

In [8]:
# 构建多轮对话

from transformers import GenerationConfig

def generate_text(prompt, model, tokenizer, **kwargs):
    # 自动匹配模型所在设备（支持多GPU/CPU）
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)
    
    # 默认生成配置（可被kwargs覆盖）
    generation_config = GenerationConfig(
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id,  # 避免Pad错误
        **kwargs  # 允许自定义参数
    )
    
    # 生成文本
    outputs = model.generate(
        **inputs,
        generation_config=generation_config,
    )
    
    # 解码并清理结果
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

class ChatBot:
    def __init__(self, model, tokenizer, max_history=5):
        """
        Args:
            model: 已加载的预训练模型
            tokenizer: 匹配模型的分词器
            max_history: 最大历史对话轮数
        """
        self.history = []
        self.max_history = max_history
        self.model = model
        self.tokenizer = tokenizer
    
    def chat(self, query, **generation_kwargs):
        # 拼接历史对话
        prompt = "\n".join([f"用户：{q}\n助手：{a}" for q, a in self.history[-self.max_history:]])
        prompt += f"\n用户：{query}\n助手："
        
        # 生成响应（传入模型和分词器）
        response = generate_text(
            prompt,
            model=self.model,
            tokenizer=self.tokenizer,
            **generation_kwargs
        )
        
        # 更新历史
        self.history.append((query, response))
        return response


In [13]:
# # 使用示例
# bot = ChatBot()
# print(bot.chat("你好，能介绍一下你自己吗？"))
# print(bot.chat("你擅长哪些领域？"))
# print(bot.chat("在你擅长的领域，人类可以做什么，不可以做什么？"))


# 初始化对话机器人
bot = ChatBot(model=model, tokenizer=tokenizer, max_history=5)

# 多轮对话测试
dialogue = [
    ("你好，能介绍一下你自己吗？", {}),
    # ("你擅长哪些领域？", {"temperature": 0.5}),  # 降低随机性
    ("你擅长哪些领域？", {}),  # 降低随机性
    ("在这些领域，人类能做什么，不能做什么？", {}),
    # ("如何预防机器学习中的过拟合？", {"max_new_tokens": 512}),  # 允许更长回答
    ("如何预防机器学习中的过拟合？", {}),  # 允许更长回答
]

for query, params in dialogue:
    print(f"用户：{query}")
    response = bot.chat(query, **params)
    print(f"助手：{response}\n")

用户：你好，能介绍一下你自己吗？
助手：
用户：你好，能介绍一下你自己吗？
助手：当然可以！我是DeepSeek-R1-Lite-2,一个由深度求索公司开发的智能助手。我拥有超过五年的技术积累，并且在多个领域中展现出色的能力和经验。

关于我的专业技能：
您好！我是DeepSeek-R1-Lite-2。如果您有任何问题或需要帮助的地方，请随时告诉我，我会尽力为您提供详细的解答。

感谢您提问！

用户：你擅长哪些领域？
助手：用户：你好，能介绍一下你自己吗？
助手：
用户：你好，能介绍一下你自己吗？
助手：当然可以！我是DeepSeek-R1-Lite-2,一个由深度求索公司开发的智能助手。我拥有超过五年的技术积累，并且在多个领域中展现出色的能力和经验。

关于我的专业技能：
您好！我是DeepSeek-R1-Lite-2。如果您有任何问题或需要帮助的地方，请随时告诉我，我会尽力为您提供详细的解答。

感谢您提问！
用户：你擅长哪些领域？
助手：您好！我是DeepSeek-R1-Lite-2，很高兴为您服务！

我可以回答以下相关的问题：

### 你的详细说明

您好！我是DeepSeek-R1-Lite-2，这是一个AI助手，它由深度求索（DeepSeek）公司开发，旨在提供专业的信息查询、内容创作和对话生成等服务。我们专注于解决复杂的技术问题，促进创新与成长。

如果你有具体的问题或者需求，欢迎提出。

用户：在这些领域，人类能做什么，不能做什么？
助手：用户：你好，能介绍一下你自己吗？
助手：
用户：你好，能介绍一下你自己吗？
助手：当然可以！我是DeepSeek-R1-Lite-2,一个由深度求索公司开发的智能助手。我拥有超过五年的技术积累，并且在多个领域中展现出色的能力和经验。

关于我的专业技能：
您好！我是DeepSeek-R1-Lite-2。如果您有任何问题或需要帮助的地方，请随时告诉我，我会尽力为您提供详细的解答。

感谢您提问！
用户：你擅长哪些领域？
助手：用户：你好，能介绍一下你自己吗？
助手：
用户：你好，能介绍一下你自己吗？
助手：当然可以！我是DeepSeek-R1-Lite-2,一个由深度求索公司开发的智能助手。我拥有超过五年的技术积累，并且在多个领域中展现出色的能力和经验。

关于我的专业技能：
您好！我是DeepSeek-R1-